In [298]:
import pandas as pd
from pymongo import MongoClient
import pandas as pd 
import numpy as np
import requests #used for HHTP requests and API calls
import re

In [299]:
df_test = pd.read_csv(r"Resources\test.csv",low_memory=False)
df_train = pd.read_csv(r"Resources\train.csv", low_memory=False)

## Clean df_train



In [300]:
# Dropping unnecessary columns
df_train.drop(columns=['ID', 'SSN', 'Num_Credit_Card', 'Interest_Rate', 'Monthly_Inhand_Salary'], inplace=True)
df_test.drop(columns=['ID', 'SSN', 'Num_Credit_Card', 'Interest_Rate', 'Monthly_Inhand_Salary'], inplace=True)

In [301]:
# Concatenate the dataframes
pd.set_option('display.max_columns', None)
df_train = pd.concat([df_train,df_test], sort= False).reset_index(drop=True)
df_train.head()

,Customer_ID,Month,Name,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS_0xd40,January,Aaron Maashoh,23,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,CUS_0xd40,February,Aaron Maashoh,23,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,CUS_0xd40,March,Aaron Maashoh,-500,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,CUS_0xd40,April,Aaron Maashoh,23,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,CUS_0xd40,May,Aaron Maashoh,23,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [302]:
# Change the dataframes column names from uppercase to lowercase and added plurals
df_train.rename(columns={"Customer_ID": "customer_id", 
"Month": "month", 
"Name": "name", 
"Age": "age",
"Occupation": "occupation",
"Annual_Income": "annual_income",
"Num_Bank_Accounts": "num_bank_accounts",
"Num_of_Loan": "num_of_loans",
"Type_of_Loan": "type_of_loans",
"Delay_from_due_date": "delay_from_due_date",
"Num_of_Delayed_Payment": "num_of_delayed_payments",
"Changed_Credit_Limit": "changed_credit_limit",
"Num_Credit_Inquiries": "num_credit_inquiries",
"Credit_Mix": "credit_mix",
"Outstanding_Debt": "outstanding_debt",
"Credit_Utilization_Ratio": "credit_utilization_ratio",
"Credit_History_Age": "credit_history_age",
"Payment_of_Min_Amount": "payment_of_min_amount",
"Total_EMI_per_month": "total_emi_per_month",
"Amount_invested_monthly": "amount_invested_monthly",
"Payment_Behaviour": "payment_behaviour",
"Monthly_Balance": "monthly_balance",
"Credit_Score": "credit_score"
}, inplace=True)
    

In [303]:
# Fix values in columns and convert to float
columns_to_clean = ['age', 'num_of_loans','annual_income', 'outstanding_debt', 'credit_utilization_ratio', 
                    'changed_credit_limit','num_of_delayed_payments', 'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance']

# Define a function to fix values with underscores in a numeric string
def fix_underscore(value):
    if isinstance(value, str) and re.search(r'\d', value):  # Check if value contains digits
        # Replace all underscores with '' and try to convert to float
        fixed_value = re.sub(r'_', '', value)
        fixed_value = re.sub(r'(?<!^)-', '', fixed_value)
        try:
            return float(fixed_value)
        except ValueError:
            return value  # Return original if conversion fails
    return value

# Apply the function to specified columns
df_train[columns_to_clean] = df_train[columns_to_clean].applymap(fix_underscore)

C:\Users\Jennifer\AppData\Local\Temp\ipykernel_5620\2204940857.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_train[columns_to_clean] = df_train[columns_to_clean].applymap(fix_underscore)


In [304]:
# Specify the columns to convert into int
columns_to_convert = ['age', 'num_bank_accounts', 'num_of_loans', 'delay_from_due_date']

# Convert the specified columns from float to int
df_train[columns_to_convert] = df_train[columns_to_convert].astype(int)

In [305]:
# First find how many customers there are, based off of cumter_id 
print(f"The number of unique customers is: {df_train.customer_id.nunique()}") 

The number of unique customers is: 12500


In [306]:
# Check what columns have null values
# Count null values in the specified columns
 
columns_to_check = ['customer_id',
    'month',
    'name',
    'age',
    'occupation',
    'annual_income',
    'num_bank_accounts',
    'num_of_loans',
    'type_of_loans',
    'delay_from_due_date',
    'num_of_delayed_payments',
    'changed_credit_limit',
    'num_credit_inquiries',
    'credit_mix',
    'outstanding_debt',
    'credit_utilization_ratio',
    'credit_history_age',
    'payment_of_min_amount',
    'total_emi_per_month',
    'amount_invested_monthly',
    'payment_behaviour',
    'monthly_balance',
    'credit_score'
]

null_counts = df_train[columns_to_check].isnull().sum()

# Show null counts
print(null_counts)

# Filter rows with null values in the specified columns
null_rows = df_train[df_train[columns_to_check].isnull().any(axis=1)]

# Show rows with null values
print(null_rows)


customer_id                     0
month                           0
name                        15000
age                             0
occupation                      0
annual_income                   0
num_bank_accounts               0
num_of_loans                    0
type_of_loans               17112
delay_from_due_date             0
num_of_delayed_payments     10500
changed_credit_limit            0
num_credit_inquiries         3000
credit_mix                      0
outstanding_debt                0
credit_utilization_ratio        0
credit_history_age          13500
payment_of_min_amount           0
total_emi_per_month             0
amount_invested_monthly      6750
payment_behaviour               0
monthly_balance              1762
credit_score                50000
dtype: int64
       customer_id      month            name   age occupation  annual_income  \
1        CUS_0xd40   February   Aaron Maashoh    23  Scientist       19114.12   
4        CUS_0xd40        May   Aaron Maash

Clean name column 
- it has null values

In [307]:
# Check if the dataframe's customer_id matches the name section
df_train.customer_id.nunique() == df_train.name.nunique()

False

In [308]:
# Find out how many names are in the dataframe 
print(f"The number of unique names is: {df_train.name.nunique()}")
print(f"The number of unique names doesn't match the number of unique customers")

The number of unique names is: 10139
The number of unique names doesn't match the number of unique customers


In [309]:
# Match the customer_id with the name
name_mapping = df_train[df_train['name'].notnull()].set_index('customer_id')['name'].to_dict()

# Fill in the name column where it is null based on the customer_id mapping
df_train['name'] = df_train.apply(
    lambda row: name_mapping.get(row['customer_id'], row['name']),
    axis=1
)

In [310]:
#Double check if number of name matches customer_id
most_common_names = df_train.groupby('customer_id')['name'].agg(lambda x: x.value_counts().idxmax()).to_dict()
print(len(most_common_names))

print(f"Number of names matches customer_id")

12500
Number of names matches customer_id


Clean age column

In [311]:
# Match the customer_id with the age
age_mapping = df_train[df_train['age'].notnull()].set_index('customer_id')['age'].to_dict()

# Fill in the age column where it is null based on the customer_id mapping
df_train['age'] = df_train.apply(
    lambda row: age_mapping.get(row['customer_id'], row['age']),
    axis=1
)

Clean occupation column

In [312]:
# Fill in the null values in the occupation column based on the customer_id mapping
loan_no_mapping = df_train[df_train['num_of_loans'].notnull()].set_index('customer_id')['num_of_loans'].to_dict()
df_train['num_of_loans'] = df_train.apply(
    lambda row: loan_no_mapping.get(row['customer_id'], row['num_of_loans']),
    axis=1
)

In [313]:
# Replace '_______' with 'N/A'
df_train['occupation'].replace('_______', "N/A", inplace=True)

C:\Users\Jennifer\AppData\Local\Temp\ipykernel_5620\4119396071.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['occupation'].replace('_______', "N/A", inplace=True)


In [314]:
# Verify that there is one occupation per customer, length should be no. of customers (12500)
occupation_counts = df_train.groupby('customer_id')['occupation'].nunique()
occupation_counts

customer_id
CUS_0x1000    1
CUS_0x1009    2
CUS_0x100b    2
CUS_0x1011    1
CUS_0x1013    1
             ..
CUS_0xff3     1
CUS_0xff4     2
CUS_0xff6     1
CUS_0xffc     2
CUS_0xffd     2
Name: occupation, Length: 12500, dtype: int64

In [315]:
print("Length is no. of customers (12500), therfore ocupation per customer matches their customer id")

Length is no. of customers (12500), therfore ocupation per customer matches their customer id


Clean num_bank_accounts column

In [316]:
df_train['num_bank_accounts'] = pd.to_numeric(df_train['num_bank_accounts'], errors='coerce')

Clean num_of_loans column

In [317]:
# Create a mapping of the maximum positive 'num_of_loans' for each customer_id
loan_positive_mapping = (
    df_train[df_train['num_of_loans'] > 0]
    .groupby('customer_id')['num_of_loans']
    .max()
    .to_dict()
)

# Update the 'num_of_loans' with positive values based on customer_id
df_train['num_of_loans'] = df_train.apply(
    lambda row: loan_positive_mapping.get(row['customer_id'], row['num_of_loans'])
    if row['num_of_loans'] < 0 else row['num_of_loans'],
    axis=1
)

Cleaning columns based off of unnecessary rows

In [318]:
# Dropping unnecessary rows
drop_na = df_train.dropna()
drop_other= drop_na[~drop_na.isin(['!@9#%8']).any(axis=1)]
df_train = drop_other[~drop_other.isin(['_']).any(axis=1)]

Verify there are no null values in the df_train

In [319]:
null_counts = df_train[columns_to_check].isnull().sum()

# Show null counts
print(null_counts)

# Filter rows with null values in the specified columns
null_rows = df_train[df_train[columns_to_check].isnull().any(axis=1)]

# Show rows with null values
print(null_rows)

customer_id                 0
month                       0
name                        0
age                         0
occupation                  0
annual_income               0
num_bank_accounts           0
num_of_loans                0
type_of_loans               0
delay_from_due_date         0
num_of_delayed_payments     0
changed_credit_limit        0
num_credit_inquiries        0
credit_mix                  0
outstanding_debt            0
credit_utilization_ratio    0
credit_history_age          0
payment_of_min_amount       0
total_emi_per_month         0
amount_invested_monthly     0
payment_behaviour           0
monthly_balance             0
credit_score                0
dtype: int64
Empty DataFrame
Columns: [customer_id, month, name, age, occupation, annual_income, num_bank_accounts, num_of_loans, type_of_loans, delay_from_due_date, num_of_delayed_payments, changed_credit_limit, num_credit_inquiries, credit_mix, outstanding_debt, credit_utilization_ratio, credit_history_age,

Clean delay_from_due_date column

In [320]:
# Replace negative values in 'delay_from_due_date' with 0

df_train['delay_from_due_date'] = df_train['delay_from_due_date'].apply(lambda x: max(x, 0))

C:\Users\Jennifer\AppData\Local\Temp\ipykernel_5620\1995175689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['delay_from_due_date'] = df_train['delay_from_due_date'].apply(lambda x: max(x, 0))


Clean changed_credit_limit column

In [ ]:
# Convert the column from object to int
df_train['changed_credit_limit'] = df_train['changed_credit_limit'].astype(int)

Clean num_credit_inquiries column

In [322]:
df_train.loc[:, 'num_credit_inquiries'] = df_train['num_credit_inquiries'].astype(int)

Clean columns that require .round(2)

In [323]:
columns_to_round = ['annual_income', 'changed_credit_limit', 'outstanding_debt', 'credit_utilization_ratio', 
                    'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance']
df_train.loc[:,columns_to_round] = df_train[columns_to_round].round(2)
pd.set_option('display.max_columns', None)

Check column types

In [324]:
df_train.dtypes

customer_id                  object
month                        object
name                         object
age                           int64
occupation                   object
annual_income               float64
num_bank_accounts             int32
num_of_loans                  int64
type_of_loans                object
delay_from_due_date           int64
num_of_delayed_payments     float64
changed_credit_limit          int32
num_credit_inquiries        float64
credit_mix                   object
outstanding_debt            float64
credit_utilization_ratio    float64
credit_history_age           object
payment_of_min_amount        object
total_emi_per_month         float64
amount_invested_monthly     float64
payment_behaviour            object
monthly_balance             float64
credit_score                 object
dtype: object

# Cleaned df_train

In [325]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
# Reset the index
df_train.reset_index(drop=True, inplace=True)
# Display the DataFrame
df_train

,customer_id,month,name,age,occupation,annual_income,num_bank_accounts,num_of_loans,type_of_loans,delay_from_due_date,num_of_delayed_payments,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,CUS_0xd40,April,Aaron Maashoh,24,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4.0,6,4.0,Good,809.98,31.38,22 Years and 4 Months,No,49.57,199.46,Low_spent_Small_value_payments,223.45,Good
1,CUS_0xd40,July,Aaron Maashoh,24,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8.0,11,4.0,Good,809.98,22.54,22 Years and 7 Months,No,49.57,178.34,Low_spent_Small_value_payments,244.57,Good
2,CUS_0x21b1,January,Rick Rothackerj,28,N/A,34847.84,2,1,Credit-Builder Loan,3,4.0,5,2.0,Good,605.03,24.46,26 Years and 7 Months,No,18.82,104.29,Low_spent_Small_value_payments,470.69,Standard
3,CUS_0x21b1,February,Rick Rothackerj,28,Teacher,34847.84,2,1,Credit-Builder Loan,7,1.0,7,2.0,Good,605.03,38.55,26 Years and 8 Months,No,18.82,40.39,High_spent_Large_value_payments,484.59,Good
4,CUS_0x21b1,April,Rick Rothackerj,28,Teacher,34847.84,2,1,Credit-Builder Loan,3,3.0,5,2.0,Good,605.03,39.18,26 Years and 10 Months,No,18.82,99.31,Low_spent_Medium_value_payments,465.68,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50107,CUS_0x8600,January,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",35,26.0,18,9.0,Bad,3571.70,22.90,5 Years and 8 Months,Yes,60.96,43.37,High_spent_Large_value_payments,328.66,Poor
50108,CUS_0x8600,February,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,23,9.0,Bad,3571.70,39.77,5 Years and 9 Months,Yes,12112.00,148.28,Low_spent_Small_value_payments,273.75,Poor
50109,CUS_0x8600,March,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,18,9.0,Bad,3571.70,24.71,5 Years and 10 Months,Yes,60.96,76.28,Low_spent_Small_value_payments,345.74,Poor
50110,CUS_0x8600,August,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,18,9.0,Bad,3571.70,37.14,6 Years and 3 Months,Yes,60.96,34.66,High_spent_Large_value_payments,337.36,Standard


In [ ]:
# Save cleaned dataframe

df_train.to_csv("Resources\cleaned_data.csv", index = False)